In [39]:
import pandas as pd
from sklearn.feature_selection import RFE
import numpy as np
from imblearn.over_sampling import SMOTE
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import detrend
import sys
import seaborn as sns
import sklearn
from mlxtend.plotting import plot_learning_curves
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.datasets import make_classification
from matplotlib import pyplot
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import random
import scipy.stats as stats
import statsmodels.api as sm
from mlxtend.plotting import plot_learning_curves
from sklearn.model_selection import learning_curve
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
from sklearn.metrics import roc_auc_score
# Import all the tools we need

# Regular EDA (exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from numpy import std
# we want our plots to appear inside the notebook
%matplotlib inline
from matplotlib.pyplot import figure
import json
import pickle
# Models from Scikit-Learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import validation_curve
# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn.model_selection import learning_curve #LearningCurveDisplay, ShuffleSplit
from sklearn import model_selection
from scipy import stats
import scikit_posthocs as sp
from scipy.stats import f_oneway
import xlwt

In [40]:
#Read
df= pd.read_csv('/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/DATASET.csv')

#Clean
    #Drop irrelevant column
df.drop(df.columns[[0]], axis=1, inplace=True) #unnamed
 
    #Save column names
column_list = list(df.columns)


#Target & features
# Feature 
X = df.drop("Category Encoded (0 = Non Fallers | 1 = Fallers)", axis=1)
X = X.values#MinMaxScaler().fit(X).transform(X)#
# Target variable
y = df["Category Encoded (0 = Non Fallers | 1 = Fallers)"].values


# Initial TT split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1,stratify=y)

In [41]:
LR = pd.read_csv("/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Models/LR.csv")
LR.drop(LR.columns[[0]], axis=1, inplace=True) #unnamed
KNN = pd.read_csv("/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Models/KNN.csv")
KNN.drop(KNN.columns[[0]], axis=1, inplace=True) #unnamed
SVM = pd.read_csv("/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Models/SVM.csv")
SVM.drop(SVM.columns[[0]], axis=1, inplace=True) #unnamed
# RF = pd.read_csv("/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Models/RF.csv")
# RF.drop(RF.columns[[0]], axis=1, inplace=True) #unnamed
#Merge all
models_list= [LR, KNN, SVM]
models=pd.concat(models_list, axis= 0)
models = models.reset_index(drop=True)
models = models.loc[:, ['auroc_test', 'accuracy_test', 'specificity_test',
       'sensitivity_test (recall)', 'precision_test', 'PPV_test', 'NPV_test']]
models = pd.DataFrame(np.around(models.values*100, 2))
models['Algorithm'] = ['LR', 'KNN', 'SVM']
models = models.loc[:, ['Algorithm', 0, 1, 2, 3, 4, 5, 6]]
models.columns = ['Algorithm', "AUROC", 'Accuracy', 'Specificity', 'Sensitivity/Recall', 'PPV/Precision', 'PPV', 'NPV']
models=models.loc[:, ['Algorithm', "AUROC", 'Accuracy', 'Specificity', 'Sensitivity/Recall', 'PPV/Precision', 'NPV']]
models

,Algorithm,AUROC,Accuracy,Specificity,Sensitivity/Recall,PPV/Precision,NPV
0,LR,98.89,97.98,97.78,100.00,81.82,97.78
1,KNN,71.67,93.94,98.89,44.44,80.00,98.89
2,SVM,96.67,93.94,93.33,100.00,60.00,93.33


In [42]:
with open('/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Models/Saved_Models/KNN_Model.pkl', 'rb') as file:  
    KNN = pickle.load(file)


with open('/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Models/Saved_Models/LogisticRegression_Model.pkl', 'rb') as file:  
    LR = pickle.load(file)

with open('/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Models/Saved_Models/SVM_Model.pkl', 'rb') as file:  
    SVM = pickle.load(file)


outcome = []
model_names = []
models = [('LogReg', LR), 
          ('SVM', SVM), 
          ('KNN', KNN)]

for model_name, model in models:
    k_fold_validation = model_selection.RepeatedKFold(n_splits=10, n_repeats=5)
    results = model_selection.cross_val_score(model, X, y, scoring='roc_auc',cv =k_fold_validation)#cv =k_fold_validation
    outcome.append(results)
    model_names.append(model_name)
    output_message = "%s| Mean=%f STD=%f" % (model_name, results.mean(), results.std())
    # print(output_message)

outcome[0] = outcome[0][~np.isnan(outcome[0])] #LR
outcome[1] = outcome[1][~np.isnan(outcome[1])] #SVM
outcome[2] = outcome[2][~np.isnan(outcome[2])] #KNN
# fig = plt.figure()
# fig.suptitle('Machine Learning Model Comparison')
# ax = fig.add_subplot(111)
# plt.boxplot(outcome)
# ax.set_xticklabels(model_names)
# plt.show()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1153: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(
/Library/Frameworks/Python.f

[LibSVM].......*........*
optimization finished, #iter = 4339
obj = -1.508573, rho = -9.781724
nSV = 147, nBSV = 133
Total nSV = 147
.........*....*.*
optimization finished, #iter = 4104
obj = -1.532879, rho = -3.950568
nSV = 153, nBSV = 141
Total nSV = 153
...
*...*...*
optimization finished, #iter = 2634
obj = -1.527120, rho = -4.982289
nSV = 147, nBSV = 131
Total nSV = 147
...
*.*.......*
optimization finished, #iter = 3047
obj = -1.357568, rho = -0.534477
nSV = 130, nBSV = 118
Total nSV = 130
..
*...*.....*
optimization finished, #iter = 3031
obj = -1.517817, rho = -4.301623
nSV = 147, nBSV = 134
Total nSV = 147
....
*...*.......*....*
optimization finished, #iter = 6246
obj = -1.700030, rho = 4.584312
nSV = 182, nBSV = 163
Total nSV = 182
[LibSVM].....*....*
optimization finished, #iter = 2736
obj = -1.562013, rho = -3.482570
nSV = 152, nBSV = 136
Total nSV = 152
...*......*
optimization finished, #iter = 2562
obj = -1.515921, rho = -4.333604
nSV = 146, nBSV = 133
Total nSV = 146


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 384, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_ranking.py

[LibSVM]..
*..*...
*....*
optimization finished, #iter = 2888
obj = -1.589790, rho = -1.227091
nSV = 155, nBSV = 145
Total nSV = 155
.....*.*
optimization finished, #iter = 1963
obj = -1.537266, rho = -0.861347
nSV = 155, nBSV = 134
Total nSV = 155
...
*.
*...
*
optimization finished, #iter = 1979
obj = -1.419130, rho = 1.915682
nSV = 138, nBSV = 126
Total nSV = 138
........*.....*.*
optimization finished, #iter = 3727
obj = -1.606138, rho = -1.572578
nSV = 163, nBSV = 147
Total nSV = 163
.....*.....*..*
optimization finished, #iter = 3628
obj = -1.479522, rho = -3.339732
nSV = 147, nBSV = 128
Total nSV = 147
....*.
*..*
optimization finished, #iter = 2596
obj = -1.713586, rho = 1.952895
nSV = 184, nBSV = 167
Total nSV = 184
[LibSVM]....
*....*.
*.*
optimization finished, #iter = 2751
obj = -1.470162, rho = -0.800374
nSV = 142, nBSV = 127
Total nSV = 142
......*..*.
*.
*
optimization finished, #iter = 2826
obj = -1.569829, rho = -3.483160
nSV = 152, nBSV = 141
Total nSV = 152
....
*
op

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 384, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_ranking.py

....
*........*.....*
optimization finished, #iter = 5973
obj = -1.750328, rho = 3.535926
nSV = 189, nBSV = 168
Total nSV = 189
[LibSVM]....
*...*
optimization finished, #iter = 2204
obj = -1.487695, rho = -2.138585
nSV = 145, nBSV = 129
Total nSV = 145
....
*..*.*
optimization finished, #iter = 1840
obj = -1.413919, rho = -2.831490
nSV = 136, nBSV = 122
Total nSV = 136
......*..*...*.*
optimization finished, #iter = 3362
obj = -1.444701, rho = -3.876829
nSV = 146, nBSV = 130
Total nSV = 146
.....
*
optimization finished, #iter = 1614
obj = -1.501151, rho = -6.090888
nSV = 145, nBSV = 130
Total nSV = 145
...
*.
*
optimization finished, #iter = 1296
obj = -1.520314, rho = -5.490720
nSV = 144, nBSV = 132
Total nSV = 144
...
*.*
optimization finished, #iter = 1704
obj = -1.669437, rho = 2.942841
nSV = 175, nBSV = 156
Total nSV = 175
[LibSVM]......*...*........*
optimization finished, #iter = 4738
obj = -1.596744, rho = -3.558847
nSV = 157, nBSV = 144
Total nSV = 157
.......*........*
opti

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 384, in _score
    return self._sign * self._score_func(y, y_pred, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_ranking.py

In [43]:
#0 LR 1 SVM 2 KNN

# Conduct the Kruskal-Wallis Test
Kruskal_res = stats.kruskal(outcome[0], outcome[1], outcome[2])

In [44]:
print(Kruskal_res, "As the p-value is less than 0.05, we can reject the null hypothesis that the median AUROC score is the same for all models. Hence, We have sufficient proof to claim that the different types of models trained lead to statistically significant differences in the AUROC scores.")

KruskalResult(statistic=121.23804339956345, pvalue=4.715126811498674e-27) As the p-value is less than 0.05, we can reject the null hypothesis that the median AUROC score is the same for all models. Hence, We have sufficient proof to claim that the different types of models trained lead to statistically significant differences in the AUROC scores.


In [45]:
post_hoc_res = sp.posthoc_dunn([outcome[0], outcome[1], outcome[2]], p_adjust = 'bonferroni')
post_hoc_res = pd.DataFrame(np.around(post_hoc_res.values, 4))
post_hoc_res.columns = ['LR', 'SVM', 'KNN']
post_hoc_res.index = ['LR', 'SVM', 'KNN']
display(post_hoc_res, 'LR and SVM auroc scores are not significantly different')

,LR,SVM,KNN
LR,1.0,1.0,0.0
SVM,1.0,1.0,0.0
KNN,0.0,0.0,1.0


'LR and SVM auroc scores are not significantly different'

In [46]:
df.head()

,Area (m^2)_fc,Range AP (m)_fc,Range ML (m)_fc,Sway Length AP (m)_fc,Sway Length ML (m)_fc,Mean Velocity AP (m/s)_fc,Mean Velocity ML (m/s)_fc,RMS Acc AP (g)_fc,RMS Acc ML (g)_fc,Area (m^2)_fo,...,Height,Weight,BMI,Falls12m,Best_T,FES_T,AgeGroup(0=young|1=old),Gender(0=F|1=M),IPAQ_S,Category Encoded (0 = Non Fallers | 1 = Fallers)
0,0.002768,0.048666,0.068331,0.403609,0.311438,0.013454,0.010381,0.023151,0.016520,0.000798,...,159.3,57.05,22.481439,1.0,19.0,7.0,1.0,0.0,1.0,1
1,0.000847,0.035414,0.026080,0.261720,0.244060,0.008724,0.008135,0.015370,0.015024,0.000524,...,152.5,51.30,22.058586,0.0,16.0,16.0,1.0,0.0,2.0,0
2,0.000358,0.024530,0.015978,0.169033,0.173541,0.005634,0.005785,0.013957,0.014463,0.000458,...,151.3,67.45,29.464841,0.0,18.0,7.0,1.0,0.0,2.0,0
3,0.000249,0.016100,0.016532,0.174627,0.118343,0.005821,0.003945,0.013938,0.011659,0.000441,...,163.3,66.35,24.881042,0.0,22.0,10.0,0.0,0.0,2.0,0
4,0.000425,0.024708,0.017979,0.259776,0.208147,0.008659,0.006938,0.018249,0.015940,0.000333,...,169.8,63.20,21.920058,0.0,23.0,8.0,0.0,1.0,2.0,0


In [47]:
col_list = list(df.columns)
col_list.remove('Category Encoded (0 = Non Fallers | 1 = Fallers)')
col_list = ['Area (m^2)_fc',
 'Range AP (m)_fc',
 'Range ML (m)_fc',
 'Sway Length AP (m)_fc',
 'Sway Length ML (m)_fc',
 'Mean Velocity AP (m/s)_fc',
 'Mean Velocity ML (m/s)_fc',
 'RMS Acc AP (g)_fc',
 'RMS Acc ML (g)_fc',
 'Area (m^2)_fo',
 'Range AP (m)_fo',
 'Range ML (m)_fo',
 'Sway Length AP (m)_fo',
 'Sway Length ML (m)_fo',
 'Mean Velocity AP (m/s)_fo',
 'Mean Velocity ML (m/s)_fo',
 'RMS Acc AP (g)_fo',
 'RMS Acc ML (g)_fo',
 'Area (m^2)_gc',
 'Range AP (m)_gc',
 'Range ML (m)_gc',
 'Sway Length AP (m)_gc',
 'Sway Length ML (m)_gc',
 'Mean Velocity AP (m/s)_gc',
 'Mean Velocity ML (m/s)_gc',
 'RMS Acc AP (g)_gc',
 'RMS Acc ML (g)_gc']

In [48]:
fallers = df.loc[(df['Category Encoded (0 = Non Fallers | 1 = Fallers)']== 1)]
fallers = fallers.drop("Category Encoded (0 = Non Fallers | 1 = Fallers)", axis=1)
nonfallers = df.loc[(df['Category Encoded (0 = Non Fallers | 1 = Fallers)']== 0)]
nonfallers = nonfallers.drop("Category Encoded (0 = Non Fallers | 1 = Fallers)", axis=1)

In [49]:
round(mean(nonfallers[col_list[0]]), 5)

0.00086

In [50]:

fallvsno_pval = []
nonfall_mean_std = []
fall_mean_std = []
for i in range(len(col_list)):
    fallvsno_pval.append(round(f_oneway(nonfallers[col_list[i]], fallers[col_list[i]])[1], 5))
    nonfall_mean_std.append(f"{round(mean(nonfallers[col_list[i]]), 5)} (+/- {round(std(nonfallers[col_list[i]]), 5)})")
    fall_mean_std.append(f"{round(round(mean(fallers[col_list[i]]), 5)+round(std(fallers[col_list[i]]), 5),5)}")
    # nonfall_mean_std.append(f">")
    # fall_mean_std.append(f"<")
ANTHROPOTAB_df = pd.DataFrame(
    {
    # f'Fallers (mean+/- std - n={len(fallers)})': fall_mean_std,
     f'Pathological Threshold (= mean(faller) + std(faller)) → Faller ≥ Pathological Threshold > Non-Faller': fall_mean_std,
    #  'P-value': fallvsno_pval
    })
ANTHROPOTAB_df.index = col_list
# ANTHROPOTAB_df.to_excel('/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Thresholdsnew.xls')
ANTHROPOTAB_df

/var/folders/rm/fz861vwx62vcc5c49ww298480000gq/T/ipykernel_7403/2972668658.py:17: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  ANTHROPOTAB_df.to_excel('/Users/cyrilleetude/Desktop/BureauMacBookAirdeCyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Thresholdsnew.xls')


,Pathological Threshold (= mean(faller) + std(faller)) → Faller ≥ Pathological Threshold > Non-Faller
Area (m^2)_fc,0.00252
Range AP (m)_fc,0.06047
Range ML (m)_fc,0.0566
Sway Length AP (m)_fc,0.44116
Sway Length ML (m)_fc,0.36988
Mean Velocity AP (m/s)_fc,0.0147
Mean Velocity ML (m/s)_fc,0.01233
RMS Acc AP (g)_fc,0.02312
RMS Acc ML (g)_fc,0.01904
Area (m^2)_fo,0.00192
